# spark 기본개념 잡기 :RDD
+ 여러 분사논드에 걸쳐 저장하는 변경불가 데이터 집합을 의미
+ RDD 생성은 직접 만들거나 파일을 통해 생성할 수 있음
+ Rdd는 transformation과 action으로 구성
    - 기존 Rdd의 데이터를 토대로 새로운 RDD를 만들어냄
    - RDD 기반으로 무언가를 계산해서 결과를 만들어 냄
+ RDD는 Lazy 로딩 방식을 사용

In [4]:
lines = sc.textFile('/usr/share/spark/README.md')

In [5]:
lines

/usr/share/spark/README.md MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [6]:
lines.collect()[:10]

['# Apache Spark',
 '',
 'Spark is a fast and general cluster computing system for Big Data. It provides',
 'high-level APIs in Scala, Java, Python, and R, and an optimized engine that',
 'supports general computation graphs for data analysis. It also supports a',
 'rich set of higher-level tools including Spark SQL for SQL and DataFrames,',
 'MLlib for machine learning, GraphX for graph processing,',
 'and Spark Streaming for stream processing.',
 '',
 '<http://spark.apache.org/>']

In [7]:
lines.count()

104

In [9]:
# 읽어들인 라인들 중에서 Spark 라는 단어를 찾음

filterLines = lines.filter(lambda x: "Spark" in x)

filterLines.collect()[:10]

['# Apache Spark',
 'Spark is a fast and general cluster computing system for Big Data. It provides',
 'rich set of higher-level tools including Spark SQL for SQL and DataFrames,',
 'and Spark Streaming for stream processing.',
 'You can find the latest Spark documentation, including a programming',
 '## Building Spark',
 'Spark is built using [Apache Maven](http://maven.apache.org/).',
 'To build Spark and its example programs, run:',
 '["Building Spark"](http://spark.apache.org/docs/latest/building-spark.html).',
 'For general development tips, including info on developing Spark using an IDE, see ["Useful Developer Tools"](http://spark.apache.org/developer-tools.html).']

## Lazy 로딩방식 확인

In [11]:
# 잘못된 위치의 파일을 읽어들일려고 시도
# 현재위치 : /home/hadoop

lines = sc.textFile("usr/share/spark/README.md")

In [ ]:
# 잘못된 경로로 오류발생 - 이전 라인에서 오류가 출력되어야 하는데
# lazy loading으로 인해 action이 호출되어야 비로소 오류가 출력

lines.collect()[:10]

## RDD 생성
+ 직접 생성한 데이터로 만들거나
    - sc.parallelize(리스트)
+ 외부 데이터로 만드는 방법 존재
    - sc.textFile(경로/파일)

In [15]:
rdd = sc.parallelize(['Hello, World!!', 'Hello, Python!!', 'this is RDD'])

rdd.count()

3

### 직책별 사원수를 조회하는 spark rdd 코드를 작성해보기
+ employees.csv를 이용

In [6]:
emp = sc.textFile('employees.csv')

emp.collect()[:5]

['EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID',
 '100,Steven,King,SKING,515.123.4567,2003-06-17,AD_PRES,24000,,,90',
 '101,Neena,Kochhar,NKOCHHAR,515.123.4568,2005-09-21,AD_VP,17000,,100,90',
 '102,Lex,De Haan,LDEHAAN,515.123.4569,2001-01-13,AD_VP,17000,,100,90',
 '103,Alexander,Hunold,AHUNOLD,590.423.4567,2006-01-03,IT_PROG,9000,,102,60']

In [7]:
# 헤더를 제외하고 데이터만 골라냄

header = emp.first()

emp = emp.filter(lambda x: header != x)

In [8]:
emp.first()

'100,Steven,King,SKING,515.123.4567,2003-06-17,AD_PRES,24000,,,90'

In [9]:
# 사원 데이터에서 , 로 각 컬럼을 분리(split)한 후
# 이름과 직책을 추출해냄

emp2 = emp.map(lambda x: (x.split(',')[6], x.split(',')[1]))

emp2.collect()[:5]

[('AD_PRES', 'Steven'),
 ('AD_VP', 'Neena'),
 ('AD_VP', 'Lex'),
 ('IT_PROG', 'Alexander'),
 ('IT_PROG', 'Bruce')]

In [10]:
# 추출한 데이터에서 직책을 1로 매핑함
maps = emp2.mapValues(lambda x: 1)
maps.collect()[:5]

[('AD_PRES', 1), ('AD_VP', 1), ('AD_VP', 1), ('IT_PROG', 1), ('IT_PROG', 1)]

In [13]:
# 같은 직책끼리 모아서 집계처리함 (reduce)
reduces = maps.reduceByKey(lambda x, y: x + y)
reduces.collect()

[('AD_PRES', 1),
 ('AD_VP', 2),
 ('IT_PROG', 5),
 ('FI_MGR', 1),
 ('FI_ACCOUNT', 5),
 ('PU_MAN', 1),
 ('PU_CLERK', 5),
 ('ST_MAN', 5),
 ('ST_CLERK', 20),
 ('SA_MAN', 5),
 ('SA_REP', 30),
 ('SH_CLERK', 20),
 ('AD_ASST', 1),
 ('MK_MAN', 1),
 ('MK_REP', 1),
 ('HR_REP', 1),
 ('PR_REP', 1),
 ('AC_MGR', 1),
 ('AC_ACCOUNT', 1)]

### 타이타닉 승객의 생존자/사망자 수를 조회하는 spark rdd 코드를 작성하세요
+ titanic.csv를 이용함

In [14]:
titanic = sc.textFile('titanic.csv')

titanic.collect()[:5]

['pclass,survived,name,sex,age,sibsp,parch,ticket,fare,embarked,life,seat,port',
 '1,1,"Allen, Miss. Elisabeth Walton",female,29.0,0,0,24160,211.3375,S,live,1st,cherbourg',
 '1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.55,S,live,1st,cherbourg',
 '1,0,"Allison, Miss. Helen Loraine",female,2.0,1,2,113781,151.55,S,dead,1st,cherbourg',
 '1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0,1,2,113781,151.55,S,dead,1st,cherbourg']

In [15]:
header = titanic.first()

titanic = titanic.filter(lambda x: header != x)

In [16]:
titanic.first()

'1,1,"Allen, Miss. Elisabeth Walton",female,29.0,0,0,24160,211.3375,S,live,1st,cherbourg'

In [17]:
titanic2 = titanic.map(lambda x: (x.split(',')[1], x.split(',')[2]))

titanic2.collect()[:5]

[('1', '"Allen'),
 ('1', '"Allison'),
 ('0', '"Allison'),
 ('0', '"Allison'),
 ('0', '"Allison')]

In [20]:
maps1 = titanic2.mapValues(lambda x: 1)
maps1.collect()[:5]

[('1', 1), ('1', 1), ('0', 1), ('0', 1), ('0', 1)]

In [21]:
titanic1 = maps1.reduceByKey(lambda x, y: x + y)
titanic1.collect()

[('1', 498), ('0', 808)]